# Chapitre 1 : Histoire de la Cryptographie

**Cours de Sécurité Informatique** - Partie 1 : Cryptographie à Clé Secrète

---

## Objectifs

Ce notebook explore l'évolution historique de la cryptographie, des chiffres classiques à la cryptographie moderne :

1. **Chiffres de substitution** : César, affine
2. **Chiffres polyalphabétiques** : Vigenère
3. **Cryptanalyse** : Analyse de fréquence, attaque Kasiski
4. **Transition vers la cryptographie moderne**

---

In [ ]:
import string
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np

## 1. Chiffre de César

Le **chiffre de César** est l'un des plus anciens chiffres connus. Il consiste à décaler chaque lettre de l'alphabet d'un nombre fixe de positions.

### Définition mathématique

Soit $k \in \{0, 1, \ldots, 25\}$ la clé (décalage) :

$$
\text{Enc}_k(m) = (m + k) \mod 26
$$

$$
\text{Dec}_k(c) = (c - k) \mod 26
$$

In [ ]:
def cesar_encrypt(plaintext: str, shift: int) -> str:
    """
    Chiffrement César.
    
    Args:
        plaintext: Texte en clair (lettres majuscules)
        shift: Décalage (0-25)
    
    Returns:
        Texte chiffré
    """
    result = []
    for char in plaintext.upper():
        if char.isalpha():
            # Convertir A-Z en 0-25
            num = ord(char) - ord('A')
            # Appliquer le décalage
            encrypted_num = (num + shift) % 26
            # Reconvertir en lettre
            encrypted_char = chr(encrypted_num + ord('A'))
            result.append(encrypted_char)
        else:
            result.append(char)
    return ''.join(result)

def cesar_decrypt(ciphertext: str, shift: int) -> str:
    """Déchiffrement César."""
    return cesar_encrypt(ciphertext, -shift)

In [ ]:
# Exemple
plaintext = "HELLO WORLD"
shift = 3

ciphertext = cesar_encrypt(plaintext, shift)
decrypted = cesar_decrypt(ciphertext, shift)

print(f"Plaintext:  {plaintext}")
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted:  {decrypted}")

### ⚠️ Sécurité du chiffre de César

**Problème fondamental** : L'espace de clés est **très petit** (seulement 26 possibilités).

**Attaque par force brute** : Essayer toutes les clés possibles !

In [ ]:
def cesar_brute_force(ciphertext: str):
    """Attaque par force brute sur le chiffre de César."""
    print("🔓 Attaque par force brute:\n")
    for shift in range(26):
        decrypted = cesar_decrypt(ciphertext, shift)
        print(f"Shift {shift:2d}: {decrypted}")

# Exemple d'attaque
secret_message = "KHOOR ZRUOG"
print(f"Message chiffré: {secret_message}\n")
cesar_brute_force(secret_message)

## 2. Chiffre de Vigenère

Le **chiffre de Vigenère** (16ème siècle) améliore César en utilisant une **clé répétée** de plusieurs lettres.

### Fonctionnement

Soit $k = k_0 k_1 \ldots k_{l-1}$ la clé de longueur $l$ :

$$
c_i = (m_i + k_{i \bmod l}) \mod 26
$$

In [ ]:
def vigenere_encrypt(plaintext: str, key: str) -> str:
    """
    Chiffrement Vigenère.
    
    Args:
        plaintext: Texte en clair
        key: Clé (mot)
    
    Returns:
        Texte chiffré
    """
    plaintext = plaintext.upper()
    key = key.upper()
    result = []
    key_index = 0
    
    for char in plaintext:
        if char.isalpha():
            # Position dans l'alphabet
            p = ord(char) - ord('A')
            k = ord(key[key_index % len(key)]) - ord('A')
            # Chiffrement
            c = (p + k) % 26
            result.append(chr(c + ord('A')))
            key_index += 1
        else:
            result.append(char)
    
    return ''.join(result)

def vigenere_decrypt(ciphertext: str, key: str) -> str:
    """Déchiffrement Vigenère."""
    ciphertext = ciphertext.upper()
    key = key.upper()
    result = []
    key_index = 0
    
    for char in ciphertext:
        if char.isalpha():
            c = ord(char) - ord('A')
            k = ord(key[key_index % len(key)]) - ord('A')
            p = (c - k) % 26
            result.append(chr(p + ord('A')))
            key_index += 1
        else:
            result.append(char)
    
    return ''.join(result)

In [ ]:
# Exemple
plaintext = "ATTACK AT DAWN"
key = "SECRET"

ciphertext = vigenere_encrypt(plaintext, key)
decrypted = vigenere_decrypt(ciphertext, key)

print(f"Plaintext:  {plaintext}")
print(f"Key:        {key}")
print(f"Ciphertext: {ciphertext}")
print(f"Decrypted:  {decrypted}")

## 3. Cryptanalyse : Analyse de Fréquence

L'**analyse de fréquence** est une technique qui exploite le fait que dans une langue donnée, certaines lettres apparaissent plus fréquemment que d'autres.

### Fréquences en français

Les lettres les plus fréquentes en français sont : **E, A, S, I, N, T, R**

In [ ]:
def frequency_analysis(text: str) -> dict:
    """Analyse de fréquence des lettres."""
    # Garder seulement les lettres
    text = ''.join(c for c in text.upper() if c.isalpha())
    
    # Compter les occurrences
    counter = Counter(text)
    total = sum(counter.values())
    
    # Calculer les fréquences en %
    frequencies = {char: (count / total) * 100 
                   for char, count in counter.items()}
    
    return dict(sorted(frequencies.items(), key=lambda x: x[1], reverse=True))

def plot_frequencies(frequencies: dict, title: str = "Fréquences"):
    """Visualisation des fréquences."""
    plt.figure(figsize=(12, 4))
    chars = list(frequencies.keys())
    freqs = list(frequencies.values())
    
    plt.bar(chars, freqs, color='steelblue')
    plt.xlabel('Lettre')
    plt.ylabel('Fréquence (%)')
    plt.title(title)
    plt.grid(axis='y', alpha=0.3)
    plt.show()

In [ ]:
# Fréquences théoriques du français
french_freq = {
    'E': 14.7, 'A': 7.6, 'S': 7.9, 'I': 7.5, 'N': 7.1, 'T': 7.2,
    'R': 6.6, 'U': 6.3, 'L': 5.5, 'O': 5.4, 'D': 3.7, 'C': 3.3,
    'P': 3.0, 'M': 3.0, 'V': 1.6, 'Q': 1.3, 'F': 1.1, 'B': 0.9,
    'G': 0.9, 'H': 0.7, 'J': 0.5, 'X': 0.4, 'Y': 0.3, 'Z': 0.1, 'K': 0.0, 'W': 0.0
}

plot_frequencies(french_freq, "Fréquences théoriques (français)")

In [ ]:
# Analyse d'un texte chiffré avec César
plaintext = """LA CRYPTOGRAPHIE EST UNE SCIENCE FASCINANTE QUI PERMET 
DE PROTEGER LES COMMUNICATIONS ET LES DONNEES"""

ciphertext = cesar_encrypt(plaintext, 7)

print("Texte chiffré:")
print(ciphertext)
print("\nAnalyse de fréquence:")

freq = frequency_analysis(ciphertext)
plot_frequencies(freq, "Fréquences du texte chiffré (César)")

# Afficher les 5 lettres les plus fréquentes
print("\nTop 5 lettres les plus fréquentes:")
for i, (char, f) in enumerate(list(freq.items())[:5], 1):
    print(f"{i}. {char}: {f:.1f}%")

### 💡 Déduction de la clé

Si la lettre la plus fréquente dans le ciphertext est **L**, et que **E** est la plus fréquente en français, alors :

$$
k = (L - E) \mod 26 = (11 - 4) \mod 26 = 7
$$

## 4. Attaque sur Vigenère : Méthode de Kasiski

La **méthode de Kasiski** (1863) permet de casser Vigenère en trouvant la **longueur de la clé**.

### Principe

1. Rechercher les **répétitions** dans le ciphertext
2. Calculer les **distances** entre ces répétitions
3. Le **PGCD** de ces distances révèle probablement la longueur de la clé
4. Une fois la longueur connue, appliquer l'analyse de fréquence sur chaque "sous-chiffre" (César)

In [ ]:
import math
from collections import defaultdict

def find_repeats(ciphertext: str, min_length: int = 3):
    """Trouve les séquences répétées dans le ciphertext."""
    ciphertext = ''.join(c for c in ciphertext.upper() if c.isalpha())
    repeats = defaultdict(list)
    
    # Chercher toutes les séquences de longueur >= min_length
    for length in range(min_length, min(len(ciphertext) // 2, 10)):
        for i in range(len(ciphertext) - length + 1):
            sequence = ciphertext[i:i+length]
            # Chercher cette séquence ailleurs
            for j in range(i + length, len(ciphertext) - length + 1):
                if ciphertext[j:j+length] == sequence:
                    distance = j - i
                    repeats[sequence].append(distance)
    
    return {seq: dist for seq, dist in repeats.items() if len(dist) > 0}

def kasiski_examination(ciphertext: str):
    """Méthode de Kasiski pour trouver la longueur de la clé."""
    repeats = find_repeats(ciphertext)
    
    print("Séquences répétées trouvées:")
    all_distances = []
    for seq, distances in list(repeats.items())[:10]:  # Top 10
        print(f"  {seq}: distances = {distances}")
        all_distances.extend(distances)
    
    if not all_distances:
        print("\nAucune répétition trouvée.")
        return None
    
    # Calculer le PGCD de toutes les distances
    gcd_result = all_distances[0]
    for d in all_distances[1:]:
        gcd_result = math.gcd(gcd_result, d)
    
    print(f"\nPGCD des distances: {gcd_result}")
    print(f"Longueur probable de la clé: {gcd_result}")
    return gcd_result

In [ ]:
# Exemple d'attaque Kasiski
plaintext = """LA CRYPTOGRAPHIE EST VRAIMENT UNE SCIENCE FASCINANTE 
QUI PERMET DE PROTEGER LES COMMUNICATIONS ET LES DONNEES 
CONTRE LES ATTAQUES MALVEILLANTES"""

key = "SECURITY"
ciphertext = vigenere_encrypt(plaintext, key)

print(f"Clé utilisée: {key} (longueur: {len(key)})")
print(f"\nCiphertext:\n{ciphertext}")
print("\n" + "="*50)
print("Attaque de Kasiski:")
print("="*50 + "\n")

key_length = kasiski_examination(ciphertext)

## 5. Transition vers la Cryptographie Moderne

### Pourquoi les chiffres classiques sont-ils cassés ?

1. **Espace de clés trop petit** (César : 26 clés)
2. **Patterns préservés** (fréquences, répétitions)
3. **Pas de confusion ni diffusion** (concepts de Shannon)
4. **Vulnérables à l'analyse statistique**

### Principes de la cryptographie moderne

**Principes de Kerckhoffs (1883)** :

> "Un système cryptographique doit être sûr même si tout est public, sauf la clé."

**Confusion et Diffusion (Shannon, 1949)** :

- **Confusion** : Relation complexe entre clé et ciphertext
- **Diffusion** : Un bit de plaintext affecte plusieurs bits de ciphertext

### One-Time Pad : Le seul chiffre parfaitement sécurisé

Le **One-Time Pad** (OTP) est le **seul** chiffre prouvé **mathématiquement incassable** (perfect security).

**Conditions** :
1. Clé de **même longueur** que le message
2. Clé **vraiment aléatoire**
3. Clé utilisée **une seule fois**

→ Voir le notebook `01_demo_otp.ipynb` pour une implémentation complète.

## Exercices

### Exercice 1 : Cassage de César

Déchiffrez le message suivant (chiffré avec César) :

```
WKXUH FDHVDU HVW XQ FKLIIU VLPSOH PDLV YXOQHUDEOH
```

In [ ]:
# Votre code ici
mystery_cipher = "WKXUH FDHVDU HVW XQ FKLIIU VLPSOH PDLV YXOQHUDEOH"

# Méthode 1 : Force brute
cesar_brute_force(mystery_cipher)

# Méthode 2 : Analyse de fréquence
freq = frequency_analysis(mystery_cipher)
print("\nLettre la plus fréquente:", list(freq.keys())[0])

### Exercice 2 : Cassage de Vigenère

Déchiffrez le message suivant (chiffré avec Vigenère, clé de 5 lettres) :

```
VPXZG IWEVH VFASD WEUKJ KTLWE VUSDG
```

In [ ]:
# Votre code ici
vigenere_cipher = "VPXZG IWEVH VFASD WEUKJ KTLWE VUSDG"

# Étape 1 : Trouver la longueur de la clé
key_length = kasiski_examination(vigenere_cipher)

# Étape 2 : Analyse de fréquence sur chaque sous-chiffre
# ... à compléter

## Conclusion

Dans ce notebook, nous avons vu :

✅ **Chiffres classiques** : César, Vigenère  
✅ **Cryptanalyse** : Analyse de fréquence, méthode de Kasiski  
✅ **Limitations** : Pourquoi ces chiffres sont vulnérables  
✅ **Transition** : Vers la cryptographie moderne (OTP, perfect security)

**Prochaines étapes** :
- Étudier le **One-Time Pad** et la **perfect security** (notebook `01_demo_otp.ipynb`)
- Comprendre les **limitations fondamentales** de la perfect security
- Introduction à la **computational security** (Chapitre 2)

---

📚 **Références** :
- Serious Cryptography (Aumasson) - Chapitre 1
- The Joy of Cryptography (Rosulek) - Chapitre 1-2
- Histoire de la cryptographie : Simon Singh, "The Code Book"